In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import SimpleITK as sitk
# Download data to work on
from downloaddata import fetch_data as fdata

In [ ]:
from myshow import myshow, myshow3d

In [ ]:
img_T1 = sitk.ReadImage(fdata("nac-hncma-atlas2013-Slicer4Version/Data/A1_grayT1.nrrd"))
img_T2 = sitk.ReadImage(fdata("nac-hncma-atlas2013-Slicer4Version/Data/A1_grayT2.nrrd"))
img_labels = sitk.ReadImage(fdata("nac-hncma-atlas2013-Slicer4Version/Data/hncma-atlas.nrrd"))

In [ ]:
sitk.Show(img_T1)

Lets find a point in one of the lateral ventricles to use as a seed for region growning.

In [ ]:
pt = (110, 120, 132)

In [ ]:
seg = sitk.Image(img_T1.GetSize(), sitk.sitkUInt8)
seg.CopyInformation(img_T1)
seg[pt] = 1
seg = sitk.BinaryDilate(seg, 3)

In [ ]:
img_T1_255 = sitk.Cast(sitk.RescaleIntensity(img_T1), sitk.sitkUInt8)
sitk.Show(sitk.LabelOverlay(img_T1_255, seg))

In [ ]:
sitk.ConnectedThreshold?

In [ ]:
seg = sitk.ConfidenceConnected(img_T1, [pt], numberOfIterations=0, multiplier=2)
sitk.Show(sitk.LabelOverlay(img_T1_255, seg))

In [ ]:
sitk.VectorConfidenceConnected?

In [ ]:
img_multi = sitk.Compose(img_T1, img_T2)
seg = sitk.VectorConfidenceConnected(img_multi, [pt], numberOfIterations=2, multiplier=4.5)
sitk.Show(sitk.LabelOverlay(img_T1_255, seg))

In [ ]:
size = img_multi.GetSize()
myshow3d(sitk.LabelOverlay(img_T1_255,seg),yslices=range(50,size[1]-50,15), zslices=range(50,size[2]-50,15), dpi=30)

In [ ]:
for m in (2.5, 3.0, 3.5, 4.0):
    seg = sitk.VectorConfidenceConnected(img_multi, [pt], numberOfIterations=2, multiplier=m)
    myshow3d(sitk.LabelOverlay(img_T1_255,seg),yslices=range(50,size[1]-50,15), zslices=range(50,size[2]-50,15), dpi=30, title="VectorConfidence multi="+str(m))

In [ ]:
sitk.Show(sitk.LabelOverlay(img_T1_255, seg))
sitk.Show(sitk.LaplacianRecursiveGaussian(img_T1,sigma=1.0))

In [ ]:
components = (img_T1, img_T2, sitk.LaplacianRecursiveGaussian(img_T1, .5), sitk.LaplacianRecursiveGaussian(img_T1, .5))
components = [sitk.Cast(img, sitk.sitkFloat32) for img in components]
img_multi = sitk.Compose(components)